In [9]:
!pip install openai

In [1]:
OPENAI_API_KEY = ""

In [14]:
from langchain_openai import *
import json
# import openai
from openai import OpenAI

In [3]:
def get_current_weather(location,unit='fahrenheit'):
    weather_info = {
        "location":location,
        "temperatire":72,
        "unit":unit,
        "forecast":["sunny","windy"]
    }
    return json.dumps(weather_info)


In [5]:
functions = [
    {
    "name":"get_current_weather",
    "description":"get current weather in current location",
    "parameters":{
        "type":"object",
        "properties":{
            "location":{
                "type":"string",
                "description":"The city and state, e.g. San Francisco, CA"
            },
            "unit":{"type":"string","enum":["celsius","fahrenheit"]}
        },
        "required":["location"]
    },
    
}
    ]

In [6]:
messages = [
    {
        "role":"user",
        "content":"what's the weather like in Boston ? "
    }
]

In [11]:
client = OpenAI(openai_api_key = OPENAI_API_KEY)

In [29]:
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

In [30]:
response

ChatCompletion(id='chatcmpl-9HhhIMVjG10KxTwTITS9S569bNqXK', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather'), tool_calls=None))], created=1714006576, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=20, prompt_tokens=81, total_tokens=101))

In [32]:
response_message = response.choices[0].message #['message']

In [33]:
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather'), tool_calls=None)

In [36]:
json.loads(response_message.function_call.arguments)

{'location': 'Boston', 'unit': 'celsius'}

In [37]:
args = json.loads(response_message.function_call.arguments)

In [38]:
get_current_weather(json.loads(response_message.function_call.arguments))

'{"location": {"location": "Boston", "unit": "celsius"}, "temperatire": 72, "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [41]:
messages = [
    {
        "role":"user",
        "content":"Hello"
    }
]

In [44]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="auto"
)

In [45]:
response

ChatCompletion(id='chatcmpl-9HhpSiKItH2pCS6CaJBcafcfDx3Rn', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1714007082, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=10, prompt_tokens=73, total_tokens=83))

In [62]:
messages = [
    {
        "role":"user",
        "content":"Whats the weather like in Boston"
    }
]

In [63]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="auto"
)

In [64]:
print(response)

ChatCompletion(id='chatcmpl-9HiFUEr9VbUpuS9Mhd7Sc85nylQKf', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather'), tool_calls=None))], created=1714008696, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_c2295e73ad', usage=CompletionUsage(completion_tokens=15, prompt_tokens=78, total_tokens=93))


In [65]:
response.choices[0].message.function_call.arguments

'{"location":"Boston"}'

In [66]:
observation = get_current_weather(response.choices[0].message.function_call.arguments)

In [67]:
observation

'{"location": "{\\"location\\":\\"Boston\\"}", "temperatire": 72, "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [68]:
messages.append(
    {
        "role":"function",
        "name":"get_current_weather",
        "content":observation
    }
)

In [69]:
print(messages)

[{'role': 'user', 'content': 'Whats the weather like in Boston'}, {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "{\\"location\\":\\"Boston\\"}", "temperatire": 72, "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'}]


In [70]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)

In [71]:
response

ChatCompletion(id='chatcmpl-9HiFn5g6SKedjYawZXWeW7CKDywIK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in Boston is sunny and windy with a temperature of 72°F.', role='assistant', function_call=None, tool_calls=None))], created=1714008715, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_caf95bb1ae', usage=CompletionUsage(completion_tokens=17, prompt_tokens=55, total_tokens=72))